In [1]:
import requests
from zipfile import ZipFile
from io import BytesIO

url = 'https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/lateral_movement/host/covenant_dcom_iertutil_dll_hijack.zip'
zipFileRequest = requests.get(url)
zipFile = ZipFile(BytesIO(zipFileRequest.content))
datasetJSONPath = zipFile.extract(zipFile.namelist()[0])

In [2]:
import pandas as pd
from pandas.io import json

df = json.read_json(path_or_buf=datasetJSONPath, lines=True)

C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
(
df[['@timestamp','Hostname','ShareName','SubjectUserName','SubjectLogonId','IpAddress','IpPort','RelativeTargetName']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 5145)
    & (df['RelativeTargetName'].str.lower().str.endswith('internet explorer\\iertutil.dll', na=False))
    & (df['AccessMask'] == '0x2')
    & (~df['SubjectUserName'].str.endswith('$', na=False))
]
.head()
)

,@timestamp,Hostname,ShareName,SubjectUserName,SubjectLogonId,IpAddress,IpPort,RelativeTargetName
12889,2020-10-09T22:30:33.185Z,WORKSTATION6.theshire.local,\\*\C$,pgustavo,0x23e34ce,172.18.39.5,51712,Program Files\Internet Explorer\iertutil.dll


In [9]:
fileCreateDf = (
# (
df[['@timestamp','Hostname','Image','TargetFilename']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 11)
    & (df['Image'].str.lower() == 'system')
    & (df['TargetFilename'].str.lower().str.endswith('internet explorer\\iertutil.dll', na=False))
]
)

fileCreateDf['Filename'] = fileCreateDf['TargetFilename'].str.split('\\').str[-1]

fileAccessedDf = (
# (
df[['@timestamp','Hostname','ShareName','SubjectUserName','SubjectLogonId','IpAddress','IpPort','RelativeTargetName']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 5145)
    & (df['AccessMask'] == '0x2')
]
)

fileAccessedDf['Filename'] = fileAccessedDf['RelativeTargetName'].str.split('\\').str[-1]

(
pd.merge(fileCreateDf, fileAccessedDf,
    on = 'Filename', how = 'inner')
)

,@timestamp_x,Hostname_x,Image,TargetFilename,Filename,@timestamp_y,Hostname_y,ShareName,SubjectUserName,SubjectLogonId,IpAddress,IpPort,RelativeTargetName
0,2020-10-09T22:30:33.173Z,WORKSTATION6.theshire.local,System,C:\Program Files\Internet Explorer\iertutil.dll,iertutil.dll,2020-10-09T22:30:33.185Z,WORKSTATION6.theshire.local,\\*\C$,pgustavo,0x23e34ce,172.18.39.5,51712,Program Files\Internet Explorer\iertutil.dll


In [15]:
fileCreateDf = (
# (
df[['@timestamp','Hostname','Image','TargetFilename']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 11)
    & (df['Image'].str.lower() == 'system')
    & (df['TargetFilename'].str.lower().str.endswith('internet explorer\\iertutil.dll', na=False))
]
)

imageLoadedDf = (
# (
df[['@timestamp','Hostname','ImageLoaded','Image']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 7)
    & (df['Image'].str.lower().str.endswith('iexplore.exe', na=False))
    & (df['ImageLoaded'].str.lower().str.endswith('internet explorer\\iertutil.dll', na=False))
]
)

firstJoinDf = (
pd.merge(fileCreateDf, imageLoadedDf,
    left_on = 'TargetFilename', right_on = 'ImageLoaded', how = 'inner')
)

firstJoinDf['Filename'] = firstJoinDf['TargetFilename'].str.split('\\').str[-1]

fileAccessedDf = (
# (
df[['@timestamp','Hostname','ShareName','SubjectUserName','SubjectLogonId','IpAddress','IpPort','RelativeTargetName']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 5145)
    & (df['AccessMask'] == '0x2')
]
)

fileAccessedDf['Filename'] = fileAccessedDf['RelativeTargetName'].str.split('\\').str[-1]

(
pd.merge(firstJoinDf, fileAccessedDf,
    on = 'Filename', how = 'inner')
)

,@timestamp_x,Hostname_x,Image_x,TargetFilename,@timestamp_y,Hostname_y,ImageLoaded,Image_y,Filename,@timestamp,Hostname,ShareName,SubjectUserName,SubjectLogonId,IpAddress,IpPort,RelativeTargetName
0,2020-10-09T22:30:33.173Z,WORKSTATION6.theshire.local,System,C:\Program Files\Internet Explorer\iertutil.dll,2020-10-09T22:30:58.902Z,WORKSTATION6.theshire.local,C:\Program Files\Internet Explorer\iertutil.dll,C:\Program Files\Internet Explorer\iexplore.exe,iertutil.dll,2020-10-09T22:30:33.185Z,WORKSTATION6.theshire.local,\\*\C$,pgustavo,0x23e34ce,172.18.39.5,51712,Program Files\Internet Explorer\iertutil.dll
